<a href="https://colab.research.google.com/github/srish-03/basic-development-projects/blob/main/movieRe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

FileNotFoundError: ignored

In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.head(1)


In [ ]:
movies = movies[['id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def convert(obj):
  l=[]
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [ ]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['cast'][0]

In [ ]:
def convert3(obj):
  l=[]
  counter = 0
  for i in ast.literal_eval(obj):
    if counter!=3:
      l.append(i['name'])
      counter+=1
    else:
      break
  return l

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags']= movies['overview'] + movies['genres'] + movies['cast']+ movies['crew']+movies['keywords']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['id','title','tags']]

In [ ]:
import nltk

In [ ]:
!pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y =[]
  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
new_df.head()

In [ ]:
 new_df['tags']= new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
 new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
 new_df['tags']= new_df['tags'].apply(lambda x:x.lower())

TEXT VECTORISATION

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
 cv.fit_transform(new_df['tags']).toarray().shape

In [ ]:
vectors[0]

Stemming

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:6]
    for i in movies_list:
      print(new_df.iloc[i[0]].title)



In [ ]:
recommend('Gandhi')

In [ ]:
import pickle


In [ ]:
pickle.dump(new_df.to_dict(),open('moviesdict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))